<a href="https://colab.research.google.com/github/plchuaa/gordonchu/blob/master/linkedin_text_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import spacy
import os
import json
import re
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string

Preprocessing by get text from json by some simple cleaning and store at a dataframe for later use

In [0]:
def text_cleaning(text):
    if len(text) > 0:
        text = text[0]
        text = re.sub(r"\\n", " ", text)
        # text = re.sub(r"\.\.\.", " ", text)
        text = re.sub(r"\.+\.+", " ", text)
        text = re.sub(r"\s{2,}", " ", text)
        text = remove_emoji(text)
        text = remove_url(text)
        text = remove_newline(text)
    else:
        text = ""
    return text


def get_name(json_dictionairy):
    data = json_dictionairy["current"][0]
    name = re.findall(r'(?:\n +)([\S\s]+)(?:\n +Status is)', data)
    name = text_cleaning(name)
    return name


def get_info(json_dictionairy):
    data = json_dictionairy["current"][0]
    info = re.findall(r'(?:See contact info(?:\n +)*[a-z0-9 A-Z+]+(?:\n +)*)([\s\S]+)', data)
    info = text_cleaning(info)
    return info


def get_summary(json_dictionairy):
    data = json_dictionairy["summary"]
    summary = text_cleaning(data)
    return summary


def list_cleaning(data):
    lists = []
    for exp in data:
        if isinstance(exp, list):
            for e in exp:
                lists.append(text_cleaning([e]))
        else:
            lists.append(text_cleaning([exp]))

    return (" ").join(lists) if len(lists) > 0 else ""


def get_experience(json_dictionairy):
    data = json_dictionairy["experience"]
    experience = list_cleaning(data)
    return experience


def get_education(json_dictionairy):
    data = json_dictionairy["education"]
    education = list_cleaning(data)
    return education


def get_license(json_dictionairy):
    data = json_dictionairy["licenses"]
    licenses = list_cleaning(data)
    return licenses


def get_skill(json_dictionairy):
    data = json_dictionairy["skills"]
    skills = list_cleaning(data)
    return skills


def get_tech(json_dictionairy):
    data = json_dictionairy["tech"]
    tech = list_cleaning(data)
    return tech


def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_url(text):
    text = re.sub(r'[a-zA-z]+://[^\s]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'[a-zA-Z]+(\.[a-zA-Z]+)+', '', text, flags=re.MULTILINE)
    return text


def remove_newline(text):
    text = re.sub(r"\n", " ", text)
    return text


# Text Preprocessing(remove stopwords,punctuations).
nlp = spacy.load('en_core_web_sm')
# build a list of stopwords to use to filter
stopwords = list(STOP_WORDS)
stopwords.append("see more")



In [31]:
import os
from google.colab import drive

# Get data from drive
drive.mount('/content/drive',force_remount=False)
os.chdir("/content/drive/My Drive/")


# Or change os.scandir('path-to-the-linkedin-dataset')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

profile_df  = pd.DataFrame(columns=["profile_name","profile_detail", "profile_info","profile_summary","profile_experience",
                                         "profile_education","profile_license","profile_skill","profile_tech"])

with os.scandir('./linkedin/') as entries:
    for each in entries:
        file = open(each, 'r', encoding="utf-8")
        profile_data = json.load(file)

        profile_name = get_name(profile_data)
        profile_info = get_info(profile_data)
        profile_summary = get_summary(profile_data)
        profile_experience = get_experience(profile_data)
        profile_education = get_education(profile_data)
        profile_license = get_license(profile_data)
        profile_skill = get_skill(profile_data)
        profile_tech = get_tech(profile_data)
        profile_detail = profile = profile_info + profile_summary + profile_experience + profile_education + profile_license + profile_skill + profile_tech

        profile_to_row = {}
        profile_to_row["profile_name"] = profile_name
        profile_to_row["profile_info"] = profile_info
        profile_to_row["profile_summary"] = profile_summary
        profile_to_row["profile_experience"] = profile_experience
        profile_to_row["profile_education"] = profile_education
        profile_to_row["profile_skill"] = profile_skill
        profile_to_row["profile_license"] = profile_license
        profile_to_row["profile_tech"] = profile_tech
        profile_to_row["profile_detail"] = profile_detail

        profile_df = profile_df.append(profile_to_row, ignore_index=True)


In [5]:
profile_df

,profile_name,profile_detail,profile_info,profile_summary,profile_experience,profile_education,profile_license,profile_skill,profile_tech
0,Abby Tang,Business Strategist skilled in Joint Marketing...,Business Strategist skilled in Joint Marketing...,Business Strategist skilled in Joint Marketin...,"APCJ Director, Cloud Field Operations Company...","University of California, Irvine Degree Name ...",,Business Alliances Go-to-market Strategy ...,
1,Ai Shinozaki,My nick name is Ai Shinozaki. I am a PG Girl S...,My nick name is Ai Shinozaki. I am a PG Girl S...,My nick name is Ai Shinozaki. I am a PG Girl ...,PG Model Company Name ABC PG Model Dates Empl...,,,Target Orientation Reading Comprehension ...,Other Skills Sexy PG Show Sexy Girl Face & Bo...
2,Dr Alan Chow MIET MIEEE MAUSIMM,Founder of a disruptive project with strong Gl...,Founder of a disruptive project with strong Gl...,Founder of a disruptive project with strong G...,"Chairman, Investment Committee Company Name C...",Renmin University of China Degree Name Doctor...,,Investments Private Equity Corporate Fina...,Industry Knowledge Equities See 25 endorsemen...
3,Abel Cheung,Administrator in a variety of coorporate netwo...,Administrator in a variety of coorporate netwo...,Administrator in a variety of coorporate netw...,Network Architect Company Name Next Media Ani...,AccessData BootCamp & FTK training Field Of S...,Novell Certified Linux Administrator (Novell ...,Linux Apache Ubuntu,
4,ai autodata,Managing Director Company Name,,,Managing Director Company Name,,,,
...,...,...,...,...,...,...,...,...,...
95,Xin LI,I am a second-year . student at CUHK. Prior to...,I am a second-year . student at CUHK. Prior to...,I am a second-year . student at CUHK. Prior t...,"Research Intern, NLP Group@Tencent AI Lab Com...",Field Of Study Faculty of Engineering Dates ...,,,
96,Wong Loretta,Company Name Google Total Duration 4 yrs 3 m...,,,Company Name Google Total Duration 4 yrs 3 mo...,The University of Hong Kong Degree Name Bache...,,Marketing Strategy Search Engine Technology...,Industry Knowledge Business Strategy See 3 en...
97,Worldmoney Blockchain News,Company Name Worldmoney Blockchain Financial...,,,Company Name Worldmoney Blockchain Financial ...,,,,
98,Wayne Liu,Technology professional with 10 years experien...,Technology professional with 10 years experien...,Technology professional with 10 years experie...,Technology Specialist Company Name 3Revolutio...,,,Digital Printing Color Management Printer...,Industry Knowledge R&D See 2 endorsements for...


Possible direction:
1. further split the profile into parts(info + summary, exp, tech+skill+license)

2. modify ways to calculate similarity


In [0]:
# spacy tokenizer to remove stopwords and punctuation, lemma words
def spacy_tokenizer(text):
    parser = English()
    mytokens = parser(text)
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    mytokens = [word for word in mytokens if word not in stopwords]
    mytokens = [word for word in mytokens if word.isalpha() or word.isnumeric() or len(word)>1 ]
    # mytokens = [word for word in mytokens if word not in string.punctuation]
    return mytokens

In [7]:
example1 = profile_df["profile_detail"][0]
example1 = spacy_tokenizer(example1)
print(example1)


['business', 'strategist', 'skilled', 'joint', 'marketing', 'partner', 'management', 'market', 'strategy', 'channel', 'strategy', 'channel', 'sales', 'developer', 'background', 'strong', 'business', 'development', 'professional', 'bs', 'focused', 'information', 'computer', 'science', 'university', 'california', 'irvine', 'business', 'strategist', 'skilled', 'joint', 'marketing', 'partner', 'management', 'market', 'strategy', 'channel', 'strategy', 'channel', 'sales', 'developer', 'background', 'strong', 'business', 'development', 'professional', 'bs', 'focused', 'information', 'computer', 'science', 'university', 'california', 'irvine', 'apcj', 'director', 'cloud', 'field', 'operations', 'company', 'f5', 'networks', 'dates', 'employed', 'feb', '2019', 'present', 'employment', 'duration', '3', 'mos', 'location', 'hong', 'kong', 'global', 'alliance', 'partner', 'development', 'company', 'amazon', 'web', 'services', 'dates', 'employed', 'jan', '2018', 'feb', '2019', 'employment', 'duratio

Different way to get similarity matrix 

**1. spacy_tokenizer + genism(tfidf)**

2. spacy_tokenizer + sklearn tfidfvectorizer + cosine similarity

3. spacy_tokenizer + spacy similarity


In [0]:
# target_profile = input("something here: ")
target_profile = "My nick name is Ai Shinozaki. I am a PG Girl Show. I can make Sexy Walk show, Sexy Face Funny Show or whatever, I can speak Japanese, Mandarin and English. I love food, sports, music, movies. I like to travel and picnic to different countries."

In [0]:
import pprint
from gensim import corpora

# Lowercase each document, split it by white space and filter out stopwords
texts = [spacy_tokenizer(text) for text in profile_df["profile_detail"]]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]

In [0]:
dictionary = corpora.Dictionary(processed_corpus)
pprint.pprint(dictionary.token2id)

In [0]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

In [0]:
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)

In [0]:
from gensim import models

tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc) 

In [14]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf] 
lsi_model.print_topics(2)

[(0,
  '0.219*"endorsements" + 0.176*"date" + 0.149*"marketing" + 0.137*"microsoft" + 0.130*"title" + 0.124*"issued" + 0.121*"data" + 0.117*"expiration" + 0.116*"endorsement" + 0.113*"sales"'),
 (1,
  '-0.308*"date" + 0.288*"marketing" + -0.221*"credential" + -0.219*"issued" + -0.209*"expiration" + 0.177*"endorsements" + -0.168*"architect" + -0.167*"aws" + -0.160*"certified" + -0.125*"authority"')]

In [0]:
for doc, as_text in zip(corpus_lsi, profile_df["profile_detail"]):
    print(doc, as_text)

In [16]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi_model[corpus])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [17]:
index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [18]:
vec_bow = dictionary.doc2bow(spacy_tokenizer(target_profile))
vec_lsi = lsi_model[vec_bow]  # convert the query to LSI space
print(vec_lsi) # [(0, 4.325712071055434), (1, 0.10321814096913436)]

[(0, 0.22168110842854571), (1, 0.0981737243587752)]


In [19]:
sims = index[vec_lsi]  # perform a similarity query against the corpus
sims_list = list(enumerate(sims))
print(sims_list)  # print (document_number, document_similarity) 2-tuples


[(0, 0.99806106), (1, 0.9950665), (2, 0.9985498), (3, 0.54104465), (4, 0.7960783), (5, 0.9550155), (6, 0.99023056), (7, 0.99955654), (8, 0.99395895), (9, 0.6477216), (10, 0.9959041), (11, 0.9804049), (12, 0.33930454), (13, 0.86385715), (14, 0.99563074), (15, 0.95820314), (16, 0.9778053), (17, 0.554533), (18, 0.99995357), (19, 0.91056055), (20, 0.98761845), (21, 0.9772434), (22, 0.85976756), (23, 0.9966134), (24, 0.63619757), (25, 0.9917169), (26, 0.95649785), (27, 0.87727237), (28, 0.95951086), (29, 0.9875529), (30, 0.98838824), (31, 0.9999729), (32, 0.71629465), (33, 0.985718), (34, 0.99239904), (35, 0.5380999), (36, 0.8769004), (37, 0.96746606), (38, 0.86673975), (39, 0.9619413), (40, 0.97278064), (41, 0.94772995), (42, 0.9953339), (43, 0.9365268), (44, 0.99956244), (45, 0.978255), (46, 0.7930718), (47, 0.99293053), (48, 0.97106355), (49, 0.96086675), (50, 0.9997908), (51, 0.6801487), (52, 0.9321604), (53, 0.95811486), (54, 0.9984579), (55, 0.8978696), (56, 0.9980837), (57, 0.5889785

In [0]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

In [21]:
print("Top 5 similar profile: ")
for each in sims[0:5]:
    print(each,profile_df["profile_detail"][each[0]])

Top 5 similar profile: 
(31, 0.9999729)   CEO Company Name Fortel Solutions Limited Dates Employed Jan 2011 – Jun 2017 Employment Duration 6 yrs 6 mos Location Hong Kong   Director Company Name Orbrich (China) International Factors Limited Dates Employed Jan 2003 – Jul 2012 Employment Duration 9 yrs 7 mos Location Tianjin City, China Cofounder of Orbrich (China) International Factors. Responsibilities include overlooking the factoring operation and the credit decision and monitoring process and building the factoring risk management system See more See more   Senior Advisor Company Name Bain & Company Dates Employed Mar 2009 – Dec 2010 Employment Duration 1 yr 10 mos Location Beijing City, China Senior Advisor to Bain's Bank of China' IMM (Internal Market Model) Project See more   Senior Vice President Company Name American Bourses Corporation Dates Employed Aug 2000 – Dec 2002 Employment Duration 2 yrs 5 mos Location Singapore Responsible of business development of both online and off

Different way to get similarity matrix 

1. spacy_tokenizer + genism(tfidf)

**2. spacy_tokenizer + sklearn tfidfvectorizer + cosine similarity**

3. spacy_tokenizer + spacy similarity


In [0]:
documents = [(" ").join(spacy_tokenizer(text)) for text in profile_df["profile_detail"]]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Create the Document Term Matrix
count_vectorizer = TfidfVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=profile_df["profile_name"])

In [24]:
df

,000,01,02002246,10,100,10118009,11,1100,111,11558293,11901698,11939,12,121,13,130,132,13d,14,140,15,150,151,1535,16,16293507,17,175,178b,18,180,1828,18345031,18351997,18603397,1898,19,1914246,193,1974641,...,xforcecolloboration,xgd,xi,xl8ys57czgyf,xml,xtradb,xu,yahoo,yale,yan,yasen,yat,year,years,yi,yin,york,yosjdv,young,youngest,youth,youths,yr,yrs,yuan,yuilop,z01,zegal,zensoft,zhefei,zhejiang,zicklin,zjz,zone,zones,zwoop,zz,новосибирская,область,россия
profile_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abby Tang,0.000000,0.0,0.0,0.035775,0.0,0.000000,0.035775,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.036050,0.120167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ai Shinozaki,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.017017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dr Alan Chow MIET MIEEE MAUSIMM,0.000000,0.0,0.0,0.032717,0.0,0.000000,0.057254,0.0,0.000000,0.0,0.0,0.0,0.051134,0.000000,0.0,0.0,0.0,0.0,0.027766,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.014325,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.036552,0.020785,0.0,0.0,0.0,0.0,0.0,0.0,0.048072,0.0,0.021979,0.120885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abel Cheung,0.000000,0.0,0.0,0.000000,0.0,0.027155,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.018857,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.065099,0.065099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ai autodata,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xin LI,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.078844,0.0,0.0,0.0,0.000000,0.078844,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.078844,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.315378,0.093136,0.079442,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.105008,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wong Loretta,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.047926,0.063902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Worldmoney Blockchain News,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.354053,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0

In [25]:
# Compute Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
sims_matrix = cosine_similarity(df, df)
print(sims_matrix)

[[1.         0.01542482 0.20143616 ... 0.16799212 0.1476735  0.18518347]
 [0.01542482 1.         0.01511693 ... 0.01474565 0.0063966  0.02797098]
 [0.20143616 0.01511693 1.         ... 0.17325747 0.14803296 0.21950924]
 ...
 [0.16799212 0.01474565 0.17325747 ... 1.         0.08843764 0.16158089]
 [0.1476735  0.0063966  0.14803296 ... 0.08843764 1.         0.11285665]
 [0.18518347 0.02797098 0.21950924 ... 0.16158089 0.11285665 1.        ]]


Different way to get similarity matrix 

1. spacy_tokenizer + genism(tfidf)

2. spacy_tokenizer + sklearn tfidfvectorizer + cosine similarity

**3. spacy_tokenizer + spacy similarity**


In [0]:
!python -m spacy download en_core_web_lg

In [27]:
nlp = spacy.load("en_core_web_lg")
token1 = nlp("i love dog.")
token5 = nlp("i love dog")
token6 = nlp("dog love me.")
token2 = nlp("i love dog?")
token3 = nlp("i hate dog.")
token4 = nlp("cat love dog.")
token7 = nlp("rabbit like carrot.")
token1.similarity
print(token1.vector_norm,token2.vector_norm,token3.vector_norm,token4.vector_norm,token5.vector_norm,token6.vector_norm,token7.vector_norm)
# 4.431595721931874 4.507516618461838 4.391567403021321 4.6381735996661915 5.044537372524752 4.369449490570207 3.8596050346561275

4.431595721931874 4.507516618461838 4.391567403021321 4.6381735996661915 5.044537372524752 4.369449490570207 3.8596050346561275


In [28]:
nlp = spacy.load("en_core_web_lg")

# get profile detail and tokenize them with spacy
documents = [(" ").join(spacy_tokenizer(text)) for text in profile_df["profile_detail"]] 

target_profile = (" ").join(spacy_tokenizer(target_profile))
target_token = nlp(target_profile)

sims_list = []
i = 0
for profile in documents:
    profile_token = nlp(profile)
    sims_list.append((i,profile_token.similarity(target_token)))
    i += 1
print(sims_list)

/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


[(0, 0.619402740195683), (1, 0.96920621010979), (2, 0.6173353595051858), (3, 0.6025665768768006), (4, 0.3319581583953515), (5, 0.6005873206297402), (6, 0.5421640297279517), (7, 0.6655084826787423), (8, 0.5714591546394197), (9, 0.5855087389474363), (10, 0.5076313448834483), (11, 0.654229655352492), (12, 0.5482235219769287), (13, 0.5937380231184313), (14, 0.6404740515960193), (15, 0.5996352252452652), (16, 0.6208728553995376), (17, 0.5156269112564439), (18, 0.5780814876021051), (19, 0.6539198574169972), (20, 0.6371772941182153), (21, 0.5962243464937379), (22, 0.5452922446072018), (23, 0.6606592177945053), (24, 0.6528448487947502), (25, 0.6409972218834031), (26, 0.5842399998851424), (27, 0.41441893041531097), (28, 0.5130685138692994), (29, 0.6044389662431003), (30, 0.5787914821334269), (31, 0.5976204040187746), (32, 0.54852973592073), (33, 0.5686242101352627), (34, 0.6547948327343357), (35, 0.5929108779066831), (36, 0.611595633877019), (37, 0.6345778203418048), (38, 0.5870773889837978), (

In [29]:
sims_list = sorted(sims_list, key=lambda item: -item[1])
print(sims_list)

[(1, 0.96920621010979), (90, 0.706554777487361), (99, 0.6873498148382473), (82, 0.672471352296247), (96, 0.671801147121337), (47, 0.6715514910912999), (81, 0.6715271776150367), (50, 0.6661874643748352), (7, 0.6655084826787423), (56, 0.6612121279527069), (23, 0.6606592177945053), (53, 0.6601266491477856), (39, 0.6557239913772176), (42, 0.655416434470732), (34, 0.6547948327343357), (11, 0.654229655352492), (19, 0.6539198574169972), (84, 0.6534594925756819), (24, 0.6528448487947502), (75, 0.6498794730885675), (93, 0.6418761100683027), (25, 0.6409972218834031), (14, 0.6404740515960193), (49, 0.6390578332319493), (80, 0.6381913018072084), (20, 0.6371772941182153), (37, 0.6345778203418048), (69, 0.6304410747873761), (41, 0.6291908694656919), (73, 0.6289875495995138), (85, 0.6275143405596927), (95, 0.6271120876115415), (78, 0.6228722559660737), (57, 0.6213853550424092), (16, 0.6208728553995376), (0, 0.619402740195683), (94, 0.6190766296346555), (45, 0.6182281370407601), (2, 0.6173353595051858

In [30]:
print("Top 5 similar profile: ")
for each in sims_list[0:5]:
    print(each,profile_df["profile_detail"][each[0]])

Top 5 similar profile: 
(1, 0.96920621010979) My nick name is Ai Shinozaki. I am a PG Girl Show. I can make Sexy Walk show, Sexy Face Funny Show or whatever, I can speak Japanese, Mandarin and English. I love food, sports, music, movies. I like to travel and picnic to different countries.  My nick name is Ai Shinozaki. I am a PG Girl Show. I can make Sexy Walk show, Sexy Face Funny Show or whatever, I can speak Japanese, Mandarin and English. I love food, sports, music, movies. I like to travel and picnic to different countries.  PG Model Company Name ABC PG Model Dates Employed Feb 2012 – Present Employment Duration 7 yrs 3 mos  Target Orientation   Reading Comprehension   Speaking Comprehension   Sexy PG Show   Sexy Girl Face & Body   Sexy Walk Show  Other Skills Sexy PG Show Sexy Girl Face & Body Sexy Walk Show 
(90, 0.706554777487361) Mr. Shiu was the Senior Editor of Sing Tao Daily’s IT supplement, “PC Market” before he co-founded Linux Pilot Ltd. A veteran in IT news, editorials 